In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from merge_years.import_data import get_full_data

In [62]:
raw_data = get_full_data('../raw_data')
raw_data

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,yellow_cards,GW,season,position,dreamteam_yearly_average,team_id,team_name,opponent_level,team_level,kickoff_date
0,aaron connolly,0,0,1,0,0.1,534,37,1,0,...,0,4,20,FWD,NaN,4.0,Brighton,1330.0,1010.0,2019-08-31
1,aaron connolly,0,0,1,0,0.3,534,43,1,0,...,0,5,20,FWD,NaN,4.0,Brighton,1180.0,1050.0,2019-09-14
2,aaron connolly,0,0,1,0,4.8,534,58,0,0,...,0,6,20,FWD,NaN,4.0,Brighton,1090.0,1010.0,2019-09-21
3,aaron connolly,0,0,2,0,0.6,534,63,1,0,...,0,7,20,FWD,NaN,4.0,Brighton,1220.0,1010.0,2019-09-28
4,aaron connolly,0,3,53,1,23.8,534,72,0,2,...,0,8,20,FWD,NaN,4.0,Brighton,1240.0,1050.0,2019-10-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114987,ørjan nyland,0,0,0,0,0.0,35,340,0,0,...,0,35,21,GK,0.0,2.0,Aston Villa,1320.0,1100.0,2021-05-09
114988,ørjan nyland,0,0,0,0,0.0,35,180,0,0,...,0,35,21,GK,0.0,2.0,Aston Villa,1130.0,1100.0,2021-05-13
114989,ørjan nyland,0,0,0,0,0.0,35,355,0,0,...,0,36,21,GK,0.0,2.0,Aston Villa,1090.0,1160.0,2021-05-16
114990,ørjan nyland,0,0,0,0,0.0,35,367,0,0,...,0,37,21,GK,0.0,2.0,Aston Villa,1190.0,1160.0,2021-05-19


In [52]:
raw_data.columns

Index(['name', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
       'element', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW', 'season', 'position',
       'dreamteam_yearly_average', 'team_id', 'team_name', 'opponent_level',
       'team_level', 'kickoff_date'],
      dtype='object')

In [63]:
#raw_data = raw_data.drop(columns=['fixture', 'element', 'team_a_score','team_h_score', 'team_id', 'team_name'], inplace=True)

# Data

## Split dataset

### Train and Test

In [64]:
j=0
for i,row in raw_data.iterrows():
    if row['name']=='caglar söyüncü':
        print(j)
        raw_data['position'].at[i]='DEF'
        j+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [65]:
raw_data['position'].unique()

array(['FWD', 'DEF', 'MID', 'GK'], dtype=object)

In [66]:
ra = 3

test_df = raw_data[(raw_data.season > 20) & (raw_data.GW >= 29-ra)]
train_df = raw_data[~((raw_data.season > 20) & (raw_data.GW >= 29-ra))]

test_df.shape, train_df.shape

((9188, 40), (105804, 40))

### Train and Test Position datasets

In [67]:
test_df_FWD = test_df[(test_df.position == 'FWD')]
test_df_DEF = test_df[(test_df.position == 'DEF')]
test_df_MID = test_df[(test_df.position == 'MID')]
test_df_GK = test_df[(test_df.position == 'GK')]

test_df_FWD.shape, test_df_DEF.shape, test_df_MID.shape, test_df_GK.shape

((1187, 40), (3204, 40), (3735, 40), (1062, 40))

In [68]:
train_df_FWD = train_df[(train_df.position == 'FWD')]
train_df_DEF = train_df[(train_df.position == 'DEF')]
train_df_MID = train_df[(train_df.position == 'MID')]
train_df_GK = train_df[(train_df.position == 'GK')]

train_df_FWD.shape, train_df_DEF.shape, train_df_MID.shape, train_df_GK.shape

((14907, 40), (35796, 40), (43598, 40), (11503, 40))

## Preprocessing

### Train : FWD

In [69]:
train_df_FWD.isnull().sum().sort_values(ascending=False)/len(train_df_FWD) 

dreamteam_yearly_average    0.893741
team_level                  0.665929
opponent_level              0.665929
team_name                   0.665929
team_id                     0.665929
team_a_score                0.000470
team_h_score                0.000470
transfers_out               0.000000
threat                      0.000000
total_points                0.000000
transfers_balance           0.000000
transfers_in                0.000000
name                        0.000000
value                       0.000000
was_home                    0.000000
assists                     0.000000
GW                          0.000000
season                      0.000000
position                    0.000000
yellow_cards                0.000000
selected                    0.000000
saves                       0.000000
goals_scored                0.000000
bonus                       0.000000
bps                         0.000000
clean_sheets                0.000000
creativity                  0.000000
e

In [84]:
train_df_FWD.head(2)

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,yellow_cards,GW,season,position,dreamteam_yearly_average,team_id,team_name,opponent_level,team_level,kickoff_date
0,aaron connolly,0,0,1,0,0.1,534,37,1,0,...,0,4,20,FWD,NaN,4.0,Brighton,1330.0,1010.0,2019-08-31
1,aaron connolly,0,0,1,0,0.3,534,43,1,0,...,0,5,20,FWD,NaN,4.0,Brighton,1180.0,1050.0,2019-09-14


In [78]:
# fwd features to be rolled

fwd_features=['name', 'assists', 'bps', 'creativity', 'goals_scored',
              'ict_index', 'influence','minutes','penalties_missed',
              'red_cards','selected','threat','total_points','transfers_balance',
              'value','yellow_cards','opponent_level','team_level']

# !!!!! To add: kickoff_time, was_home, GW, season

In [79]:
train_FWD = train_df_FWD[fwd_features]
train_FWD.shape

(14907, 18)

In [82]:
rolling_train_FWD = train_FWD.groupby('name').rolling(ra, closed = 'left').mean()
rolling_train_FWD

assists        bps  creativity  goals_scored  \
name                                                                   
aaron connolly 0            NaN        NaN         NaN           NaN   
               1            NaN        NaN         NaN           NaN   
               2            NaN        NaN         NaN           NaN   
               3       0.000000   1.000000    1.733333      0.000000   
               4       0.000000   1.333333    1.900000      0.000000   
...                         ...        ...         ...           ...   
álvaro negredo 114823  0.000000  13.666667    7.500000      0.333333   
               114824  0.000000  16.333333    7.500000      0.333333   
               114825  0.333333  19.666667   10.166667      0.333333   
               114826  0.333333  18.000000    9.466667      0.333333   
               114827  0.333333  17.666667   20.233333      0.333333   

                       ict_index  influence    minutes  penalties_missed  \
name                                                                       
aaron connolly 0             NaN        NaN        NaN               NaN   
               1             NaN        NaN        NaN               NaN   
               2             NaN        NaN        NaN               NaN   
               3        1.566667   1.066667  16.666667               0.0   
               4        1.600000   1.066667  17.000000               0.0   
...                          ...        ...        ...               ...   
álvaro negredo 114823   5.933333  16.600000  67.333333               0.0   
               114824   5.500000  17.133333  81.333333               0.0   
               114825   4.800000  17.533333  78.333333               0.0   
               114826   4.166667  14.666667  79.333333               0.0   
               114827   6.533333  17.000000  84.333333               0.0   

                       red_cards       selected     threat  total_points  \
name                                                                       
aaron connolly 0             NaN            NaN        NaN           NaN   
               1             NaN            NaN        NaN           NaN   
               2             NaN            NaN        NaN           NaN   
               3             0.0   12277.666667  13.000000      1.000000   
               4             0.0   23177.333333  13.000000      1.000000   
...                          ...            ...        ...           ...   
álvaro negredo 114823        0.0  564284.333333  35.000000      3.666667   
               114824        0.0  624816.333333  30.000000      4.000000   
               114825        0.0  641492.333333  20.000000      5.000000   
               114826        0.0  587912.000000  17.333333      5.000000   
               114827        0.0  526030.000000  28.000000      5.000000   

                       transfers_balance      value  yellow_cards  \
name                                                                
aaron connolly 0                     NaN        NaN           NaN   
               1                     NaN        NaN           NaN   
               2                     NaN        NaN           NaN   
               3             6226.333333  45.000000           0.0   
               4             9038.666667  45.000000           0.0   
...                                  ...        ...           ...   
álvaro negredo 114823       70565.333333  62.000000           0.0   
               114824      124403.666667  62.666667           0.0   
               114825       80565.666667  62.666667           0.0   
               114826       10483.666667  62.333333           0.0   
               114827      -62388.000000  61.666667           0.0   

                       opponent_level   team_level  
name                                                
aaron connolly 0                  NaN          NaN  
               1                  NaN          NaN  

In [83]:
rolling_train_FWD = rolling_train_FWD.reset_index()
rolling_train_FWD

,name,level_1,assists,bps,creativity,goals_scored,ict_index,influence,minutes,penalties_missed,red_cards,selected,threat,total_points,transfers_balance,value,yellow_cards,opponent_level,team_level
0,aaron connolly,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aaron connolly,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aaron connolly,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aaron connolly,3,0.000000,1.000000,1.733333,0.000000,1.566667,1.066667,16.666667,0.0,0.0,12277.666667,13.000000,1.000000,6226.333333,45.000000,0.0,1200.000000,1023.333333
4,aaron connolly,4,0.000000,1.333333,1.900000,0.000000,1.600000,1.066667,17.000000,0.0,0.0,23177.333333,13.000000,1.000000,9038.666667,45.000000,0.0,1163.333333,1023.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14902,álvaro negredo,114823,0.000000,13.666667,7.500000,0.333333,5.933333,16.600000,67.333333,0.0,0.0,564284.333333,35.000000,3.666667,70565.333333,62.000000,0.0,NaN,NaN
14903,álvaro negredo,114824,0.000000,16.333333,7.500000,0.333333,5.500000,17.133333,81.333333,0.0,0.0,624816.333333,30.000000,4.000000,124403.666667,62.666667,0.0,NaN,NaN
14904,álvaro negredo,114825,0.333333,19.666667,10.166667,0.333333,4.800000,17.533333,78.333333,0.0,0.0,641492.333333,20.000000,5.000000,80565.666667,62.666667,0.0,NaN,NaN
14905,álvaro negredo,114826,0.333333,18.000000,9.466667,0.333333,4.166667,14.666667,79.333333,0.0,0.0,587912.000000,17.333333,5.000000,10483.666667,62.333333,0.0,NaN,NaN


In [85]:
result = map(lambda gw:gw, train_df_FWD['GW'])
rolling_train_FWD['GW'] = list(result)

result_1 = map(lambda season:season, train_df_FWD['season'])
rolling_train_FWD['season'] = list(result_1)

result_2 = map(lambda home:home, train_df_FWD['was_home'])
rolling_train_FWD['was_home'] = list(result_2)

result_3 = map(lambda kickoff:kickoff, train_df_FWD['kickoff_time'])
rolling_train_FWD['kickoff_time'] = list(result_3)

In [86]:
rolling_train_FWD

,name,level_1,assists,bps,creativity,goals_scored,ict_index,influence,minutes,penalties_missed,...,total_points,transfers_balance,value,yellow_cards,opponent_level,team_level,GW,season,was_home,kickoff_time
0,aaron connolly,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4,20,False,14:00:00
1,aaron connolly,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5,20,True,14:00:00
2,aaron connolly,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6,20,False,16:30:00
3,aaron connolly,3,0.000000,1.000000,1.733333,0.000000,1.566667,1.066667,16.666667,0.0,...,1.000000,6226.333333,45.000000,0.0,1200.000000,1023.333333,7,20,False,14:00:00
4,aaron connolly,4,0.000000,1.333333,1.900000,0.000000,1.600000,1.066667,17.000000,0.0,...,1.000000,9038.666667,45.000000,0.0,1163.333333,1023.333333,8,20,True,11:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14902,álvaro negredo,114823,0.000000,13.666667,7.500000,0.333333,5.933333,16.600000,67.333333,0.0,...,3.666667,70565.333333,62.000000,0.0,NaN,NaN,34,17,True,18:45:00
14903,álvaro negredo,114824,0.000000,16.333333,7.500000,0.333333,5.500000,17.133333,81.333333,0.0,...,4.000000,124403.666667,62.666667,0.0,NaN,NaN,35,17,True,13:05:00
14904,álvaro negredo,114825,0.333333,19.666667,10.166667,0.333333,4.800000,17.533333,78.333333,0.0,...,5.000000,80565.666667,62.666667,0.0,NaN,NaN,36,17,False,19:00:00
14905,álvaro negredo,114826,0.333333,18.000000,9.466667,0.333333,4.166667,14.666667,79.333333,0.0,...,5.000000,10483.666667,62.333333,0.0,NaN,NaN,37,17,True,14:00:00


### Train : DEF